In [1]:
## Question 4.5

In [2]:
## Location 1: lambda = 3 for both returns and rentals. Poisson distribution
## Location 2: lambda = 4 for renatals, lambda = 2 for returns
## 20 cars maximum between two locations, extras are sent back to rental company
## Renting out a car gives reward of $10. Costs $2 to move a car at night
## Cars can be rented the day after they are returned
## Discount rate is .9

In [3]:
from numpy import random
import numpy as np
from math import e, factorial

class JacksCarRental:

    def __init__(self, location_a_cars, location_b_cars, rent_reward=10, move_penalty=2, lambda_rental_a=3, 
                lambda_rental_b=4, lambda_return_a=3, lambda_return_b=2, discount=0.9):
        self.location_a_cars = location_a_cars
        self.location_b_cars = location_b_cars
        self.rent_reward = rent_reward
        self.move_penalty = move_penalty
        self.lambda_rental_a = lambda_rental_a
        self.lambda_rental_b = lambda_rental_b
        self.lambda_return_a = lambda_return_a
        self.lambda_return_b = lambda_return_b
        self.discount_rate = discount


    def get_daily_demand(self):
        a_rentals = random.poisson(lam=self.lambda_rental_a, size=1)
        b_rentals = random.poisson(lam=self.lambda_rental_b, size=1)
        a_returns = random.poisson(lam=self.lambda_return_a, size=1)
        b_returns = random.poisson(lam=self.lambda_return_b, size=1)
        return a_rentals, b_rentals, a_returns, b_returns

    
    def run_day(self):
        a_rentals, b_rentals, a_returns, b_returns = self.get_daily_demand()
        if self.location_a_cars - a_rentals < 0:
            a_rentals = self.location_a_cars
        if self.location_b_cars - b_rentals < 0:
            b_rentals = self.location_b_cars

        reward = 10 * (a_rentals + b_rentals)

        if self.location_a_cars + a_returns > 20:
            self.location_a_cars = 20
        else:
            self.location_a_cars += a_returns
        if self.location_b_cars + b_returns > 20:
            self.location_b_cars = 20
        else:
            self.location_b_cars += b_returns

        return reward

    
    def rental_probability(self, rented_a, rented_b, current_a, current_b):
        p_rent_a = self.poisson_probability(self.lambda_rental_a, rented_a)
        p_rent_b = self.poisson_probability(self.lambda_rental_b, rented_b)
        p_return_a = self.poisson_probability(self.lambda_return_a, current_a - rented_a)
        p_return_b = self.poisson_probability(self.lambda_return_b, current_b - rented_b)
        return p_rent_a * p_rent_b * p_return_a * p_return_b
    
    
    def poisson_probability(self, lam, n):
        return (np.exp(-lam) * lam**n) / np.math.factorial(n)
    



    # def policy_iteration(self):
    #     ## Initialization
    #     V = np.zeros((20, 20))
    #     pi = np.ones((20, 20))  # will define as how many cars to move from A to B

    #     ## Policy Evaluation
    #     delta = 15
    #     while delta < .5:
    #         for a in V:
    #             for b in a:
    #                 if a + b > 20:
    #                     continue
    #                 v = V[a, b]
    #                 V[a, b] = [self.get_poisson_estimator(self.lambda_rental_a, x) * (x * self.rent_reward + self.discount_rate * V[a - x, b - y]) for x in range(a) for y in range(b)]


In [ ]:
def policy_evaluation(policy, env, num_iterations=100, threshold=1e-6):
    V = policy.copy()
    for i in range(num_iterations):
        delta = 0
        for a in range(21):
            for b in range(21):
                v = V[a, b]
                total_reward = 0
                for rented_a in range(a+1):
                    for rented_b in range(b+1):
                        prob = env.rental_probability(rented_a, rented_b, a, b)
                        reward = (rented_a + rented_b) * env.rent_reward
                        next_a = min(a - rented_a + env.move_penalty, 20)
                        next_b = min(b - rented_b + env.move_penalty, 20)
                        total_reward += prob * (reward + env.discount_rate * V[next_a, next_b])
                V[a, b] = total_reward
                delta = max(delta, abs(v - V[a, b]))
        if delta < threshold:
            break
    return V